# Data Cleaning
We may have to come back to this.

In [17]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-a33890a0-b99f-fc20-8e93-c56e70f0c690)


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
import pandas as pd
import numpy as np

In [20]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  # drive.mount('/content/drive')
  proj_dir = "/content/drive/MyDrive/ece884_project/"
else:
  proj_dir = ""

Using the [stroke dataset](https://trialsjournal.biomedcentral.com/articles/10.1186/1745-6215-12-101#MOESM1) we want to use these data to generate additioal samples. For a data dictionary see [the paper](https://www.researchgate.net/publication/51068185_The_International_Stroke_Trial_database).

In [21]:
df = pd.read_csv(f"{proj_dir}data_raw/ist.csv", low_memory=False)
df.columns = [x.lower() for x in df.columns]

In [22]:
df = df.applymap(lambda x: x.lower() if type(x) == str else x)
df = df.apply(lambda x: x.replace(to_replace=['n', 'y'], value=[0, 1]))
df = df.apply(lambda x: x.replace(to_replace=['c', 'u', 'unknown'], value=np.nan))

I am KNN inputing missing for contnous valued columns, then replacing with 0 the dummy variables.

In [23]:
df['sex'] = df.sex.replace(to_replace=['f', 'm'], value=[0, 1])
dummy_cols = ['rconsc', 'stype', 'rxhep', 'dmajnchx', 'dplace', 'fplace', 'nccode', 'country']
dummy_df = pd.get_dummies(df[dummy_cols])
dummy_df = dummy_df.fillna(0, inplace=True)

In [25]:
df = df.apply(pd.to_numeric, errors='coerce')
numeric_df = df.drop(columns=dummy_cols)
misc_drop = ["rdate", "dsidex", "dnostrkx", "drshd", "ddeadx", "fdeadx"] # these are not helpful and causing problems

In [26]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)

In [27]:
numeric_df = numeric_df.drop(columns=misc_drop)
numeric_df = pd.DataFrame(imputer.fit_transform(numeric_df),columns = numeric_df.columns)
df = pd.concat([numeric_df, dummy_df])

KeyboardInterrupt: ignored

In [ ]:
df.to_csv(f"{proj_dir}data_clean/clean.csv", index=False)